In [39]:
import glob
import os
import pandas as pd
import numpy as np

In [23]:
path = "/mnt/n4hukb/ns_morphologist"
subpath = "ses-2/anat/t1mri/default_acquisition/default_analysis/folds/3.1"
qc20_file = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/qc_post.tsv"
qc_file = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/qc.tsv"

In [15]:
list_folders = glob.glob(f"{path}/*")
list_folders = [x for x in list_folders]
list_subjects = [os.path.basename(f) for f in list_folders]

In [16]:
list_subjects

['sub-1049910',
 'sub-1070872',
 'sub-3751903',
 'sub-2661657',
 'sub-4418577',
 'sub-5627124',
 'sub-4732417',
 'sub-3494193',
 'sub-5063457',
 'sub-6013312',
 'sub-3576371',
 'sub-2580428',
 'sub-5650914',
 'sub-4792187',
 'sub-4493045',
 'sub-5006220',
 'sub-3943942',
 'sub-4869106',
 'sub-1684581',
 'sub-3911590',
 'sub-3240010',
 'sub-3409265',
 'sub-2421676',
 'sub-2019271',
 'sub-4766709',
 'sub-2009286',
 'sub-1389578',
 'sub-1478534',
 'sub-1213477',
 'sub-3993337',
 'sub-4772432',
 'sub-5838145',
 'sub-1916516',
 'sub-5317124',
 'sub-2059879',
 'sub-5849145',
 'sub-4421281',
 'sub-4716329',
 'sub-5100031',
 'sub-5269509',
 'sub-3885807',
 'sub-3139270',
 'sub-1601969',
 'sub-5492364',
 'sub-2577033',
 'sub-2505847',
 'sub-5786774',
 'sub-2039063',
 'sub-4884350',
 'sub-1403604',
 'sub-1297602',
 'sub-3169449',
 'sub-3257076',
 'sub-3319774',
 'sub-5557380',
 'sub-1775410',
 'sub-3758439',
 'sub-3385518',
 'sub-1064547',
 'sub-4682907',
 'sub-3313541',
 'sub-5915175',
 'sub-36

In [51]:
[x for x in list_subjects if not'sub' in x]

['bvdb']

In [17]:
qc = pd.DataFrame(list_subjects, columns=["participant_id"]).set_index("participant_id")

In [18]:
qc["qc"] = 1
qc["comments"] =  ""

In [19]:
qc.head()

,qc,comments
participant_id,,
sub-1049910,1,
sub-1070872,1,
sub-3751903,1,
sub-2661657,1,
sub-4418577,1,


In [21]:
qc["qc"] = 1
qc["comments"] =  ""

for f, s in zip(list_folders, list_subjects):
    path_to_graph = f"{f}/{subpath}"

    # Checks of left graph exists
    graph_file = f"{path_to_graph}/L{s}.arg"
    if not os.path.isfile(graph_file):
        print(f"\nL{s}.arg does not exist")
        qc.loc[s, "qc"] = 0
        qc.loc[s, "comments"] = "Left graph does not exist"

    
    graph_file = f"{path_to_graph}/R{s}.arg"
    if not os.path.isfile(graph_file):
        print(f"\nR{s}.arg does not exist")
        qc.loc[s, "qc"] = 0
        qc.loc[s, "comments"] = "Right graph does not exist"

    if qc.loc[s, "qc"] == 1:
        print('.', end='')


......................................................................
Lsub-1532046.arg does not exist

Rsub-1532046.arg does not exist
..........................................................................................................................................
Lsub-2933213.arg does not exist

Rsub-2933213.arg does not exist
............
Lsub-4499315.arg does not exist

Rsub-4499315.arg does not exist
............................................................................................................................................
Lsub-3945549.arg does not exist

Rsub-3945549.arg does not exist
.............
Lsub-5489670.arg does not exist

Rsub-5489670.arg does not exist
..............................................................................................................................
Lsub-4131670.arg does not exist

Rsub-4131670.arg does not exist
..............
Lsub-2911548.arg does not exist

Rsub-2911548.arg does not exist
.........................

In [ ]:
graph_file

'/mnt/n4hukb/ns_morphologist/sub-462091/ses-2/anat/t1mri/default_acquisition/default_analysis/folds/3.1/Rsub-462091.arg'

In [ ]:
qc["qc"].sum()

40425

In [22]:
len(qc)

42767

In [26]:
qc20 = pd.read_csv(qc20_file, index_col=0, sep='\t')

In [27]:
qc20.head()

,qc,comments
participant_id,,
sub-2859308,1,NaN
sub-5201474,1,NaN
sub-2967866,1,NaN
sub-2707712,1,NaN
sub-4770610,1,NaN


In [40]:
qc_all = qc.merge(qc20, left_index=True, right_index=True, how='outer')
qc_all.head()

,qc_x,comments_x,qc_y,comments_y
participant_id,,,,
bvdb,0,Right graph does not exist,NaN,NaN
sub-1000021,1,,1.0,NaN
sub-1000325,1,,NaN,NaN
sub-1000458,1,,1.0,NaN
sub-1000575,1,,1.0,NaN


In [41]:
qc_all[qc_all.qc_y==0].comments_y.unique()

array(['At least one void crop', 'missing R graph',
       'too many voxels (more than 1.2 x 90th quantile)',
       'problem in foldlabel generation',
       'Problem in foldlabel generation',
       'Artifact inducing wrong MNI phasing'], dtype=object)

In [43]:
qc_all["qc"] = qc_all["qc_x"]
qc_all.loc[qc_all.qc_y==0, "qc"] = 0
qc_all["comments"] = np.where(qc_all.qc_y==0, qc_all["comments_y"], qc_all["comments_x"])

In [44]:
qc_all[qc_all.qc==0].comments.unique()

array(['Right graph does not exist', 'At least one void crop',
       'missing R graph',
       'too many voxels (more than 1.2 x 90th quantile)',
       'problem in foldlabel generation',
       'Problem in foldlabel generation',
       'Artifact inducing wrong MNI phasing'], dtype=object)

In [46]:
qc_all = qc_all[["qc", "comments"]]

In [47]:
qc_all.head()

,qc,comments
participant_id,,
bvdb,0,Right graph does not exist
sub-1000021,1,
sub-1000325,1,
sub-1000458,1,
sub-1000575,1,


In [53]:
qc_all.to_csv(qc_file, sep='\t')